## **File Setup**

In [3]:
import os

# base_path = r"C:\Users\User\Downloads\Drawing sample 2"

# satellite_path = os.path.join(base_path, "OP_communitymap..tif")
# cad_path = os.path.join(base_path, "OP_ungeo.tif")
# output_gdb = os.path.join(base_path, "output.gdb")
# dwg_path = os.path.join(base_path, "10309 - Proposed Site Plan.dwg")
# tfw_path = os.path.join(base_path, "OP_ungeo.tfw")
# output_shp = os.path.join(base_path, "output_shapefile")

# base_path = r"C:\Users\User\Downloads\Drawing sample 3"

# satellite_path = os.path.join(base_path, "US_Virgin_Airpotrt_CAD_ungeoreferenced.tif")
# cad_path = os.path.join(base_path, "US_Virgin_Airpotrt_Communit.tif")
# output_gdb = os.path.join(base_path, "output.gdb")
# dwg_path = os.path.join(base_path, "Ungeoreferenced_USVI.dwg")
# tfw_path = os.path.join(base_path, "US_Virgin_Airpotrt_CAD_ungeoreferenced.tfw")
# output_shp = os.path.join(base_path, "output_shapefile")

base_path = r"C:\Users\User\Downloads\6414_Test"

satellite_path = os.path.join(base_path, "GTAA_ungeoreferenced.tif")
cad_path = os.path.join(base_path, "community.tif")
output_gdb = os.path.join(base_path, "GTAA_output.gdb")
dwg_path = os.path.join(base_path, "GTAA_cad.dwg")
tfw_path = os.path.join(base_path, "community.tfw")
output_shp = os.path.join(base_path, "GTAA_shapefile")

desc = arcpy.Describe(dwg_path)
print(desc.spatialReference.name)
print(desc.spatialReference.factoryCode)
    

NameError: name 'arcpy' is not defined

## **Georeferencing**

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import math

def preprocess_image(image, target_size=(600, 600)):
    """Reduce background noise while preserving main structures for both satellite and CAD images."""
    img_resized = cv2.resize(image, target_size)
    gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    binary = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 15, 2)
    kernel = np.ones((3, 3), np.uint8)
    clean = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)
    return img_resized, gray, clean

def add_border(image, border_size=100):
    """Add a larger border around the image to exclude areas from matching."""
    return cv2.copyMakeBorder(image, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=(255, 255, 255))

def rotate_image(image, angle):
    """Rotate image by a given angle without cropping."""
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, M, (w, h))

def match_edge_maps(edges1, edges2, method="ORB"):
    """Match keypoints using ORB or SIFT."""
    if method == "ORB":
        detector = cv2.ORB_create(nfeatures=1000)
    else:
        detector = cv2.SIFT_create()
    
    kp1, des1 = detector.detectAndCompute(edges1, None)
    kp2, des2 = detector.detectAndCompute(edges2, None)
    
    if des1 is None or des2 is None:
        return None, [], []
    
    bf = cv2.BFMatcher(cv2.NORM_HAMMING if method == "ORB" else cv2.NORM_L2, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)[:100]
    
    return matches, kp1, kp2

def calculate_angle(p1, p2):
    """Calculate the angle of the line between two points relative to the x-axis (in degrees),
    ensuring the result is between 0 and 90 degrees."""
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    angle = abs(math.atan2(dy, dx) * 180.0 / math.pi)  # Convert to degrees and ensure positive
    angle = min(angle, 180 - angle)  # Ensure it's within 0-90 degrees
    return angle

def calculate_distance(p1, p2):
    """Calculate the Euclidean distance between two points."""
    return np.linalg.norm(np.array(p2) - np.array(p1))

def filter_extreme_slope_and_long_matches(matches, kp1, kp2, max_angle=45, max_distance_ratio=0.1):
    """Filter matches to remove those with extreme slopes or excessive distances."""
    filtered_matches = []
    max_distance = 0  # To track the longest distance
    
    # First pass: calculate the maximum distance
    for match in matches:
        p1 = tuple(map(int, kp1[match.queryIdx].pt))
        p2 = tuple(map(int, kp2[match.trainIdx].pt))
        distance = calculate_distance(p1, p2)
        max_distance = max(max_distance, distance)
    
    # Second pass: filter based on slope and distance
    for match in matches:
        p1 = tuple(map(int, kp1[match.queryIdx].pt))
        p2 = tuple(map(int, kp2[match.trainIdx].pt))
        
        # Calculate the angle of the line formed by the match (p1, p2)
        angle = calculate_angle(p1, p2)
        
        # Calculate the Euclidean distance between the points
        distance = calculate_distance(p1, p2)
        
        # Keep match only if:
        # 1. The angle is less than the max_angle threshold (not too steep)
        # 2. The distance is within a certain ratio of the max distance (not too long)
        if abs(angle) <= max_angle and distance <= max_distance * max_distance_ratio:
            filtered_matches.append(match)
    
    return filtered_matches

def filter_quadrant_matches(matches, kp1, kp2, img1_shape, img2_shape):
    """Filter matches to ensure that points are within the same quadrant in both images."""
    h1, w1 = img1_shape[:2]  # Ignore the channel dimension
    h2, w2 = img2_shape[:2]  # Ignore the channel dimension
    
    valid_matches = []
    for match in matches:
        p1 = kp1[match.queryIdx].pt
        p2 = kp2[match.trainIdx].pt
        
        # Determine quadrants for both images
        quadrant1 = (p1[0] < w1 // 2, p1[1] < h1 // 2)
        quadrant2 = (p2[0] < w2 // 2, p2[1] < h2 // 2)
        
        # Keep match only if they are in the same quadrant
        if quadrant1 == quadrant2:
            valid_matches.append(match)
    
    return valid_matches

def visualize_matches(img1, kp1, img2, kp2, matches, title):
    """Draw and visualize matched keypoints side by side."""
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR) if len(img1.shape) == 2 else img1
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR) if len(img2.shape) == 2 else img2
    
    new_h, new_w = max(h1, h2), w1 + w2
    canvas = np.zeros((new_h, new_w, 3), dtype=np.uint8)
    canvas[:h1, :w1] = img1
    canvas[:h2, w1:w1+w2] = img2
    
    for match in matches:
        kp1_pt = kp1[match.queryIdx].pt
        kp2_pt = (kp2[match.trainIdx].pt[0] + w1, kp2[match.trainIdx].pt[1])
        color = tuple(random.randint(0, 255) for _ in range(3))
        cv2.line(canvas, tuple(map(int, kp1_pt)), tuple(map(int, kp2_pt)), color, 1)
    
    plt.figure(figsize=(12, 6))
    plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# satellite_path = r"C:\Users\User\Downloads\TIFF GTAA Georeferenced (1)\TIFF GTAA Georeferenced\community.tif"
# cad_path = r"C:\Users\User\Downloads\6414_Test\GTAA_ungeoreferenced.tif"
img1 = cv2.imread(satellite_path)
img2 = cv2.imread(cad_path)

# Add a larger border around the satellite image
border_size = 100
img1_with_border = add_border(img1, border_size)
img2_with_border = add_border(img2, border_size)

img1_processed, gray1, edges1 = preprocess_image(img1_with_border)
img2_processed, gray2, edges2 = preprocess_image(img2_with_border)

best_matches, best_kp1, best_kp2, best_angle = [], [], [], 0
for angle in range(0, 360, 10):
    edges2_rot = rotate_image(edges2, angle)
    matches, kp1, kp2 = match_edge_maps(edges1, edges2_rot, method="ORB")
    if matches and len(matches) > len(best_matches):
        best_matches, best_kp1, best_kp2, best_angle = matches, kp1, kp2, angle

# Filter matches to ensure they are in the same quadrant
best_matches = filter_quadrant_matches(best_matches, best_kp1, best_kp2, img1.shape, img2.shape)

# Filter for extreme slopes and long distances
best_matches = filter_extreme_slope_and_long_matches(best_matches, best_kp1, best_kp2, max_angle=45, max_distance_ratio=0.1)

#visualize_matches(img1_processed, best_kp1, img2_processed, best_kp2, best_matches, f"ORB Best Match at {best_angle}°")

matches_sift, kp1_sift, kp2_sift = match_edge_maps(edges1, edges2, method="SIFT")
if matches_sift:
    #matches_sift = filter_quadrant_matches(matches_sift, kp1_sift, kp2_sift, img1.shape, img2.shape)
    matches_sift = filter_extreme_slope_and_long_matches(matches_sift, kp1_sift, kp2_sift, max_angle=45, max_distance_ratio=0.1)
    visualize_matches(img1_processed, kp1_sift, img2_processed, kp2_sift, matches_sift, "SIFT Matches")


## **CAD to GIS Conversion**

In [ ]:
import cv2
import numpy as np
import arcpy
import os
import time
import uuid

def extract_matched_points(matches, kp1, kp2):
    """Extract (x, y) coordinates from matched keypoints."""
    cad_points = np.array([kp1[m.queryIdx].pt for m in matches])  # CAD image
    map_points = np.array([kp2[m.trainIdx].pt for m in matches])  # Map image
    return cad_points, map_points

def transform_map_point_to_geospatial(px, py, tfw_params):
    """Convert pixel coordinates (px, py) in the map image to geospatial coordinates using TFW."""
    A, D, B, E, C, F = tfw_params
    geo_x = A * px + B * py + C
    geo_y = D * px + E * py + F
    return geo_x, geo_y

def compute_dwg_bounding_box(dwg_path):
    """Extract the DWG bounding box (minX, maxX, minY, maxY) using ArcPy."""
    desc = arcpy.Describe(dwg_path)
    extent = desc.extent
    return extent.XMin, extent.XMax, extent.YMin, extent.YMax

def transform_cad_to_dwg(px, py, cad_size, dwg_bbox):
    """Convert pixel coordinates in the CAD image to DWG coordinate system."""
    minX, maxX, minY, maxY = dwg_bbox
    img_width, img_height = cad_size
    scale_x = (maxX - minX) / img_width
    scale_y = (maxY - minY) / img_height
    dwg_x = minX + px * scale_x
    dwg_y = maxY - py * scale_y  # Flip Y-axis
    return dwg_x, dwg_y

def read_tfw(tfw_path):
    with open(tfw_path, "r") as f:
        values = [float(line.strip()) for line in f.readlines()]
    return values  # [A, D, B, E, C, F]

def georeference_cad(dwg_path, control_points, output_gdb, output_shp):
    """Use control points to georeference CAD and convert to GIS format."""
    desc = arcpy.Describe(dwg_path)
    spatial_ref = arcpy.SpatialReference(26917)

    fc_name = f"control_points_{uuid.uuid4().hex[:8]}"
    control_points_fc = os.path.join(output_gdb, fc_name)
    print(control_points_fc)

    if arcpy.Exists(control_points_fc):
        arcpy.Delete_management(control_points_fc)

    print("Contents of GDB:")
    arcpy.env.workspace = output_gdb
    print(arcpy.ListFeatureClasses())

    arcpy.env.workspace = output_gdb
    result = arcpy.CreateFeatureclass_management(
        output_gdb,
        fc_name,
        "POINT",
        spatial_reference=spatial_ref
    )

    print("Feature class created at:", result)
    print("Exists immediately after creation?", arcpy.Exists(control_points_fc))
    time.sleep(5)

    print("Exists?", arcpy.Exists(control_points_fc))
    
    with arcpy.da.InsertCursor(control_points_fc, ['SHAPE@XY']) as cursor:
        for dwg_pt, geo_pt in control_points:
            print("Inserting point:", dwg_pt)
            cursor.insertRow([(dwg_pt[0], dwg_pt[1])])
    
    # Convert CAD to GIS
    arcpy.CADToGeodatabase_conversion(
        dwg_path,
        output_gdb,
        "cad_features",
        "1",  # reference scale as string
        spatial_ref
    )
    
    # Export EACH feature class inside cad_features dataset to individual shapefiles
    cad_feature_dataset = os.path.join(output_gdb, "cad_features")
    feature_classes = arcpy.ListFeatureClasses(feature_dataset="cad_features")

    if not os.path.exists(output_shp):
        os.makedirs(output_shp)

    for fc in feature_classes:
        full_fc_path = os.path.join(cad_feature_dataset, fc)
        print(f"Exporting: {fc} -> {output_shp}")
        arcpy.conversion.FeatureClassToShapefile([full_fc_path], output_shp)

    print("All CAD features exported to shapefiles.")
    
def main(dwg_path, map_image, cad_image, tfw_path, matches, kp1, kp2, output_gdb, output_shp):
    """Main function to process matches and georeference CAD to GIS."""
    # Step 1: Extract matched pixel coordinates
    cad_points, map_points = extract_matched_points(matches, kp1, kp2)
    
    # Step 2: Compute the original size of images
    cad_size = cad_image.shape[1], cad_image.shape[0]  # (width, height)
    map_size = map_image.shape[1], map_image.shape[0]
    
    # Step 3: Convert map points to geospatial coordinates
    tfw_params = read_tfw(tfw_path)
    geospatial_points_map = [transform_map_point_to_geospatial(x, y, tfw_params) for x, y in map_points]
    
    # Step 4: Compute DWG bounding box
    dwg_bbox = compute_dwg_bounding_box(dwg_path)
    
    # Step 5: Convert CAD points to DWG coordinates
    dwg_points = [transform_cad_to_dwg(x, y, cad_size, dwg_bbox) for x, y in cad_points]
     
    # Step 6: Create control points and georeference the CAD file
    control_points = list(zip(dwg_points, geospatial_points_map))
    georeference_cad(dwg_path, control_points, output_gdb, output_shp)
    
    print("Georeferencing complete. CAD converted to GIS format and shapefile exported.")


In [ ]:
if not arcpy.Exists(output_gdb):
    arcpy.CreateFileGDB_management(out_folder_path=os.path.dirname(output_gdb),
                                    out_name=os.path.basename(output_gdb))
    
desc = arcpy.Describe(dwg_path)
print(desc.spatialReference.name)
print(desc.spatialReference.factoryCode)
    
main(dwg_path = dwg_path,
     map_image=cv2.imread(satellite_path),
     cad_image=cv2.imread(cad_path),
     tfw_path= tfw_path,  # Extract these from the TFW file
     matches=matches_sift, 
     kp1=kp1_sift, 
     kp2=kp2_sift,
     output_gdb=output_gdb,
     output_shp=output_shp)


## **Quality Control (Error Detection)**

### **Feature Extraction** 

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import LineString

# NUMBER OF DATA POINTS
NUM_DATA_POINTS = 10000

print('Reading Shapefile...')

# Load the shapefile
shapefile_path = os.path.join(base_path, 'GTAA_errors', 'GTAA_errors.shp')
gdf = gpd.read_file(shapefile_path)
print('Shapefile Loaded!\n')

# exploding
gdf = gdf.explode(index_parts=True).reset_index()

class DataProcessing:
    completedCount = 0

    def remove_duplicates(self, gdf):
        gdf_temp = gdf.iloc[:, 1:].copy()
        gdf_temp = gdf_temp.drop(columns=['Error'])
        dup_mask = gdf_temp.duplicated(keep='first')

        gdf_res = gdf[~dup_mask]
        gdf_res.loc[:, 'Error'] = gdf_res['Error'].apply(lambda x: 0 if x == 2 else x)
        return gdf_res

    def compute_linestring_metrics(self, linestring):
        """
        Computes multiple metrics for a LINESTRING Z:
        - is_closed: Whether the linestring forms a closed path.
        - length_3d: The total 3D length of the linestring.
        - line_curvature: The ratio of actual path length to straight-line distance.
        - num_vertices: The number of vertices in the linestring.
        - vertex_density: The number of vertices per unit length.
        - total_angle_change: The sum of absolute angle changes between segments.
        - is_point: Whether the linestring consists of a single repeated point.
        - num_connections: Number of other lines that touch this line, but don't go through (i.e. a 3-way intersection, not a 4-way intersection)
        - num_intersections: Number of other lines that pass through this line
        - bounding_box_width: The width of the bounding box of the linestring.
        - bounding_box_height: The height of the bounding box of the linestring.
        - relative_angle_change: Ratio of total_angle_change to length of the line. (Relative Angle Change to Line Length)
        - proximity_to_neighbors: The average minimum distance to neighboring lines.
        - similarity_score: Similarity score with close neighbor lines based on selected metrics (length, curvature, vertices) and normalized based on neighbor group
        - num_neighbors: Number of neighboring lines within a radius equal to 1% of the total map size.
        """
        if not isinstance(linestring, LineString) or len(linestring.coords) < 2:
            return False, 0.0, 1.0, 0, 0.0, 0.0, True, 0, 0 ,0.0, 0.0, 0.0, 0.0, 0.0, 0 # Defaults for invalid linestrings

        coords = np.array(linestring.coords)
        first_point = coords[0]
        last_point = coords[-1]
        is_closed = np.array_equal(first_point, last_point)

        diffs = np.diff(coords, axis=0)
        segment_lengths = np.linalg.norm(diffs, axis=1)
        sum_length = np.sum(segment_lengths)

        straight_line_distance = np.linalg.norm(last_point - first_point)
        line_curvature = sum_length / straight_line_distance if straight_line_distance > 0 else 1.0

        num_vertices = len(coords)

        # Vertices per unit length
        vertex_density = num_vertices / sum_length if sum_length > 0 else 0.0

        # Compute total angle change
        if len(linestring.coords) == 2:
            total_angle_change = 0.0
        else:
            valid_segments = segment_lengths > 0
            unit_vectors = diffs[valid_segments] / segment_lengths[:, None][valid_segments]
            dot_products = np.einsum("ij,ij->i", unit_vectors[:-1], unit_vectors[1:])
            angles = np.arccos(np.clip(dot_products, -1.0, 1.0))
            # Ensure we sum all angle changes without taking absolute values
            total_angle_change = np.sum(angles)

        # Check if all coordinates are the same (i.e., single repeated point)
        is_point = np.all(coords == coords[0])

        # Compute number of connections (lines that share at least one point)
        connected_lines_idx = list(spatial_index.intersection(linestring.bounds))
        connected_lines = gdf.iloc[connected_lines_idx]
        # Subtract 1 to exclude the line itself
        num_connections = sum(connected_lines.geometry.touches(linestring)) 

        # Compute number of lines that intersect with this line
        intersecting_lines_idx = list(
            spatial_index.intersection(linestring.bounds))
        intersecting_lines = gdf.iloc[intersecting_lines_idx]
        # Subtract 1 to exclude the line itself
        num_intersections = sum(intersecting_lines.geometry.intersects(linestring)) - 1

        # Compute bounding box width and height
        minx, miny, maxx, maxy = linestring.bounds
        bounding_box_width = maxx - minx
        bounding_box_height = maxy - miny

        # Get the bounding box of the current line
        buffered_line = linestring.buffer(search_radius)
        possible_neighbors = list(spatial_index.intersection(buffered_line.bounds))

        num_neighbors = len(possible_neighbors) - 1

        # Compute proximity to neighbors (average minimum distance)
        min_distances = []
        for neighbor_idx in possible_neighbors:
            neighbor = gdf.iloc[neighbor_idx].geometry
            if neighbor != linestring:  # Avoid self-comparison
                distance = linestring.distance(neighbor)
                min_distances.append(distance)

        # If there are neighbors, compute the average distance
        proximity_to_neighbors = search_radius * 2
        # Compute weighted proximity: Closer neighbors have higher weight
        if min_distances:
            weights = [1 / d if d > 0 else 1 for d in min_distances]
            proximity_to_neighbors = np.average(min_distances, weights=weights)
        else:
            proximity_to_neighbors = search_radius * 2  # Default if no neighbors

        # Compute similarity score with neighbors
        similarity_score = 0.0
        neighbor_metrics = []
        for neighbor_idx in possible_neighbors:
            neighbor = gdf.iloc[neighbor_idx].geometry
            if neighbor != linestring:  # Avoid self-comparison
                neighbor_coords = np.array(neighbor.coords)
                neighbor_diffs = np.diff(neighbor_coords, axis=0)
                neighbor_segment_lengths = np.linalg.norm(neighbor_diffs, axis=1)
                neighbor_sum_length = np.sum(neighbor_segment_lengths)

                neighbor_first_point = coords[0]
                neighbor_last_point = coords[-1]
                neighbor_straight_line_distance = np.linalg.norm(
                    neighbor_last_point - neighbor_first_point)
                neighbor_line_curvature = neighbor_sum_length / \
                    neighbor_straight_line_distance if neighbor_straight_line_distance > 0 else 1.0
                
                neighbor_num_vertices = len(neighbor_coords)

                neighbor_metrics.append(
                    [neighbor_sum_length, neighbor_line_curvature, neighbor_num_vertices])

        # Extract and normalize metrics for the neighbor group
        lengths = np.array([metrics[0] for metrics in neighbor_metrics])
        curvatures = np.array([metrics[1] for metrics in neighbor_metrics])
        vertices = np.array([metrics[2] for metrics in neighbor_metrics])

        # Normalize the current line's metrics against the neighbor group
        length_mean, length_std = np.mean(lengths), np.std(lengths)
        curvature_mean, curvature_std = np.mean(curvatures), np.std(curvatures)
        vertices_mean, vertices_std = np.mean(vertices), np.std(vertices)

        normalized_length = (sum_length - length_mean) / \
            length_std if length_std > 0 else 0.0
        normalized_curvature = (line_curvature - curvature_mean) / \
            curvature_std if curvature_std > 0 else 0.0
        normalized_vertices = (num_vertices - vertices_mean) / \
            vertices_std if vertices_std > 0 else 0.0

        # Calculate similarity score using normalized metrics
        if (num_neighbors > 0):
            similarity_score = 1 / (1 + abs(normalized_length) + abs(normalized_curvature) +
                                abs(normalized_vertices))

        relative_angle_change = total_angle_change / sum_length if sum_length != 0 else 0

        self.completedCount += 1
        if self.completedCount % 1000 == 0:
            print(f'Finished {self.completedCount} rows')

        return is_closed, sum_length, line_curvature, num_vertices, vertex_density, total_angle_change, is_point, num_connections, num_intersections, bounding_box_width, bounding_box_height, relative_angle_change, proximity_to_neighbors, similarity_score, num_neighbors

gdf = gdf.reset_index()
gdf['id'] = gdf['index']

processing = DataProcessing()
# Shuffle the GeoDataFrame and take the top 10,000 rows
# Shuffling the GeoDataFrame
gdf = processing.remove_duplicates(gdf)

gdf1 = gdf[gdf['Error'] == 1]
gdf2 = gdf[gdf['Error'] != 1]
gdf2 = gdf2.head(NUM_DATA_POINTS)  # Take the top 10,000 rows
gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2], ignore_index=True))

gdf = gdf.sample(frac=1, random_state=42)
gdf = gdf.reset_index(drop=True)
spatial_index = gdf.sindex

print('Duplicates Removed + Random Sampling + Set Spacial Index\n')

minx, miny, maxx, maxy = gdf.total_bounds

# Compute the width and height of the bounding box
bbox_width = maxx - minx
bbox_height = maxy - miny
search_radius = 0.01 * max(bbox_width, bbox_height)

print('Computing Features...')
# Apply the function and ensure the output shape matches the expected columns
metrics = gdf.geometry.apply(processing.compute_linestring_metrics)
print('Features Computed!\n')

# Convert the result to a DataFrame and check the lengths
metrics_df = pd.DataFrame(metrics.tolist(), columns=[
                          "is_closed", "sum_length", "line_curvature", "num_vertices", "vertex_density", "total_angle_change", "is_point", "num_connections", "num_intersections", "bounding_box_width", "bounding_box_height", "relative_angle_change", "proximity_to_neighbors", "similarity_score", "num_neighbors"])

# Assign the calculated metrics to the original GeoDataFrame
gdf = gdf.join(metrics_df)

# Compute metrics for each line
columns = ["id", "is_closed", "sum_length", "line_curvature", "num_vertices", "vertex_density",
           "total_angle_change", "is_point", "num_connections", "num_intersections", "bounding_box_width", "bounding_box_height", "relative_angle_change", "proximity_to_neighbors", "similarity_score", "num_neighbors", "Error"]
gdf_features = gdf[columns]
gdf_features = gdf_features.reset_index()
print('Final Training Set Created with Labels')

output_path = os.path.join(base_path, 'GTAA_Errors_with_metrics.csv')
gdf_features.to_csv(output_path, index=False)
print('Training Set Stored')

### **Error Detection**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix

In [4]:
import os

# Load dataset
feature_path = os.path.join(base_path, "GTAA_Errors_with_metrics.csv")
df = pd.read_csv(feature_path) 

# Print dataset head for debugging
print("Dataset preview:")
print(df.head())

# Convert is_point columns from TRUE/FALSE to 1/0
df.replace({"TRUE": 1, "FALSE": 0, True: 1, False: 0}, inplace=True)

# Identify feature and target columns
ignore_cols = ["id", "index"]
target_col = "Error"
features = df.drop(columns=ignore_cols + [target_col])
target = df[target_col]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Print dataset statistics
print("\nTraining set:")
print(f"Total data points: {len(y_train)}")
print(y_train.value_counts(normalize=True) * 100)

print("\nTest set:")
print(f"Total data points: {len(y_test)}")
print(y_test.value_counts(normalize=True) * 100)

# Standardize features for SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Dataset preview:
   index      id  is_closed  sum_length  line_curvature  num_vertices  \
0      0  156838      False    5.799891        2.707294             6   
1      1    6898       True   14.995678        1.000000            17   
2      2    6941       True    7.000716        1.000000             5   
3      3     509      False    0.901641        1.000000             2   
4      4    9176      False    0.680246        1.000000             2   

   vertex_density  total_angle_change  is_point  num_connections  \
0        1.034502            5.269789     False                0   
1        1.133660           23.564956     False                0   
2        0.714213            4.712721     False                0   
3        2.218177            0.000000     False                0   
4        2.940115            0.000000     False                0   

   num_intersections  bounding_box_width  bounding_box_height  \
0                  1              2.0073               2.1411   
1    

C:\Users\User\AppData\Local\Temp\ipykernel_67240\2137943297.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"TRUE": 1, "FALSE": 0, True: 1, False: 0}, inplace=True)


### **Error Visualization**

In [5]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(n_estimators=100, random_state=42)
name = "LightGBM"
    
# For models requiring scaling (SVM, XGBoost, GradientBoosting, LightGBM)
if name in ["SVM (Polynomial Kernel)", "XGBoost", "Gradient Boosting", "LightGBM"]:
    model.fit(X_train_scaled, y_train)
    y_train_pred = model.predict(X_train_scaled)
    y_test_pred = model.predict(X_test_scaled)
else:
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

for dataset, y_true, y_pred in [("Train", y_train, y_train_pred), ("Test", y_test, y_test_pred)]:
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    percent_errors_identified = (tp / (tp + fn)) * 100 if (tp + fn) > 0 else 0
    
    print(f"\n{name} - {dataset} set:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"Percentage of Errors Correctly Identified: {percent_errors_identified:.2f}%")
    print(classification_report(y_true, y_pred))
    
# Cross-validation
cv_scores = cross_val_score(model, X_train_scaled if name in ["SVM (Polynomial Kernel)", "XGBoost", "Gradient Boosting", "LightGBM"] else X_train, y_train, cv=5, scoring='accuracy')
print(f"\n{name} - Cross-validation Accuracy (5-fold): {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

[LightGBM] [Info] Number of positive: 493, number of negative: 7990
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000254 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2645
[LightGBM] [Info] Number of data points in the train set: 8483, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.058116 -> initscore=-2.785437
[LightGBM] [Info] Start training from score -2.785437

LightGBM - Train set:
Accuracy: 1.0000
F1-score: 1.0000
False Positives: 0
False Negatives: 0
Percentage of Errors Correctly Identified: 100.00%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7990
           1       1.00      1.00      1.00       493

    accuracy                           1.00      8483
   macro avg       1.00      1.00      1.00      8483
weighted avg       1.00     

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



LightGBM - Test set:
Accuracy: 0.9995
F1-score: 0.9955
False Positives: 0
False Negatives: 1
Percentage of Errors Correctly Identified: 99.10%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2010
           1       1.00      0.99      1.00       111

    accuracy                           1.00      2121
   macro avg       1.00      1.00      1.00      2121
weighted avg       1.00      1.00      1.00      2121

[LightGBM] [Info] Number of positive: 394, number of negative: 6392
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000446 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2644
[LightGBM] [Info] Number of data points in the train set: 6786, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.058061 -> initscore=-2.786452
[LightGBM] [Info] Start training from score -2.786452
[LightGBM] [Warning] No further splits 

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 394, number of negative: 6392
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000590 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2637
[LightGBM] [Info] Number of data points in the train set: 6786, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.058061 -> initscore=-2.786452
[LightGBM] [Info] Start training from score -2.786452
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 394, number of negative: 6392
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000300 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2640
[LightGBM] [Info] Number of data points in the train set: 6786, number of used features: 14
[Lig

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 395, number of negative: 6392
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000341 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2642
[LightGBM] [Info] Number of data points in the train set: 6787, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.058199 -> initscore=-2.783917
[LightGBM] [Info] Start training from score -2.783917
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 395, number of negative: 6392
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000273 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2641
[LightGBM] [Info] Number of data points in the train set: 6787, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.058199 -> initscore=-2.783917

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [20]:
import geopandas as gpd 

feature_scaler = StandardScaler()
features_scaled = feature_scaler.fit_transform(features)

error_predictions = model.predict(features_scaled)
df["prediction"] = error_predictions

# Load the shapefile
shapefile_path = os.path.join(base_path, 'GTAA_errors', 'GTAA_errors.shp')
features_gdf = gpd.read_file(shapefile_path)
print('Shapefile Loaded!\n')

# exploding
features_gdf = features_gdf.explode(index_parts=True).reset_index()





C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Shapefile Loaded!



In [34]:
# Set df index to 'id' to enable fast matching
df_indexed = df.set_index('id')

# Optionally: keep only relevant gdf rows
gdf_filtered = gdf[gdf.index.isin(df_indexed.index)].copy()

# Add new column to gdf by mapping the 'prediction' from df
gdf_filtered['Error Prediction'] = df_indexed.loc[gdf_filtered.index, 'prediction'].values
gdf_filtered.loc[gdf_filtered['Error'] == 2, 'Error'] = 0

predicted_shpfolder= os.path.join(base_path, 'GTAA_prediction')
os.makedirs(predicted_shpfolder, exist_ok=True)
predicted_shpfile = os.path.join(predicted_shpfolder, 'GTAA_prediction.shp')
gdf_filtered.to_file(predicted_shpfile, driver="ESRI Shapefile")


gdf_filtered[gdf_filtered['Error Prediction'] == 1].head(500)

C:\Users\User\AppData\Local\Temp\ipykernel_67240\4017676373.py:14: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_filtered.to_file(predicted_shpfile, driver="ESRI Shapefile")
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Error Prediction' to 'Error Pred'
  ogr_write(


,level_0,level_1,Entity,Handle,Layer,LyrFrzn,EntColor,LyrColor,Linetype,LineWt,EntLineWt,LyrLineWt,BlkLineWt,RefName,Error,Shape_Leng,geometry,Error Prediction
635,635,0,Insert,4D5,SP-E-RNW-PV,0,-1,7,Continuous,15,-1,15,-1,XREF1,0,0.962188,"LINESTRING Z (610773.181 4837863.142 0, 610774...",1
2288,2288,0,Insert,4D5,SP-E-RNW-PV,0,-1,7,Continuous,15,-1,15,-1,XREF1,0,1.325395,"LINESTRING Z (610323.359 4837152.198 0, 610324...",1
3009,3009,0,Insert,4D5,SP-E-RNW-PV,0,-1,7,Continuous,15,-1,15,-1,XREF1,0,0.376205,"LINESTRING Z (610213.129 4837143.046 0, 610213...",1
3010,3010,0,Insert,4D5,SP-E-RNW-PV,0,-1,7,Continuous,15,-1,15,-1,XREF1,0,0.838672,"LINESTRING Z (610213.255 4837142.906 0, 610212...",1
9398,9398,0,Insert,4D5,SP-E-RNW-PV,0,-1,7,Continuous,15,-1,15,-1,XREF1,0,1.298599,"LINESTRING Z (609408.777 4837168.723 0, 609407...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157716,157716,0,Insert,43559A,0,0,-1,7,Continuous,25,-1,-3,-1,Buildings,1,1.403000,"LINESTRING Z (611343.528 4837643.454 0, 611344...",1
157722,157722,0,Insert,43559A,0,0,-1,7,Continuous,25,-1,-3,-1,Buildings,1,3.889863,"LINESTRING Z (611381.206 4837589.397 0, 611383...",1
157723,157723,0,Insert,43559A,0,0,-1,7,Continuous,25,-1,-3,-1,Buildings,1,1.049936,"LINESTRING Z (611381.637 4837589.307 0, 611381...",1
157724,157724,0,Insert,43559A,0,0,-1,7,Continuous,25,-1,-3,-1,Buildings,1,5.801748,"LINESTRING Z (611381.418 4837587.632 0, 611383...",1
